# Dataset analysis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import utils

In [2]:
df = pd.read_json("data_no_bool.json").T
df.head()

,label,time_series
0,6,"[{'BallAcceleration': 1636.7987723122642, 'Dis..."
1,-1,"[{'BallAcceleration': 0.0, 'DistanceWall': 39...."
2,-1,"[{'BallAcceleration': 124246.29375405672, 'Dis..."
3,-1,"[{'BallAcceleration': -8210.634011562943, 'Dis..."
4,-1,"[{'BallAcceleration': 1197.5360615055833, 'Dis..."


In [3]:
df.label.value_counts()

 2    60
 6    46
 5    45
-1    43
 3    38
 7    36
 1    30
Name: label, dtype: int64

----------

In [4]:
#from sklearn.utils import shuffle
import sklearn as sk
from sklearn.model_selection import train_test_split
seed = 0

# Shuffle the dataframe
shuffled_df = sk.utils.shuffle(df, random_state=seed)
shuffled_df.head()

,label,time_series
111,6,"[{'BallAcceleration': 4841.49275538644, 'Dista..."
175,6,"[{'BallAcceleration': -347.5747892882355, 'Dis..."
12,7,"[{'BallAcceleration': 0.0, 'DistanceWall': 400..."
218,1,"[{'BallAcceleration': 106808.88211978199, 'Dis..."
205,7,"[{'BallAcceleration': 14189.044826174126, 'Dis..."


In [5]:
from utils import extract_matrix
# Create the matrix version
shuffled_df.time_series = shuffled_df.time_series.apply(extract_matrix)
shuffled_df["shape"] = shuffled_df.time_series.apply(lambda x: x.shape)
shuffled_df.head()

,label,time_series,shape
111,6,"[[4841.49275538644, 3920.09, 2013.1, 164.34092...","(19, 17)"
175,6,"[[-347.5747892882355, 4002.97, 2013.0, 218.624...","(19, 17)"
12,7,"[[0.0, 4004.29, 2013.08, 193.23514638905624, 1...","(16, 17)"
218,1,"[[106808.88211978199, 13.230000000000018, 345....","(19, 17)"
205,7,"[[14189.044826174126, 3329.59, 2013.15, 153.21...","(16, 17)"


In [6]:
X_train, X_test, y_train, y_test = train_test_split(shuffled_df['time_series'], shuffled_df['label'], train_size=.75, random_state=seed)

In [7]:
len(X_train)

223

In [8]:
len(X_test)

75

In [9]:
a = y_train.value_counts()
a/=np.sum(a)
a

 2    0.219731
 5    0.161435
 6    0.161435
-1    0.134529
 3    0.125561
 7    0.103139
 1    0.094170
Name: label, dtype: float64

In [10]:
a = y_test.value_counts()
a/=np.sum(a)
a

 7    0.173333
-1    0.173333
 2    0.146667
 3    0.133333
 6    0.133333
 5    0.120000
 1    0.120000
Name: label, dtype: float64

# Baseline
Use as feature vector the mean of each field of the time series

In [11]:
X_train_mean_fv = utils.compute_mean_feature_vector(X_train)
X_test_mean_fv = utils.compute_mean_feature_vector(X_test)

In [12]:
matrix_X_train_mean_fv = utils.from_series_to_matrix(17, X_train_mean_fv)
matrix_X_test_mean_fv = utils.from_series_to_matrix(17, X_test_mean_fv)

In [13]:
prova_train = y_train.to_numpy().astype('int')
prova_test = y_test.to_numpy().astype('int')

In [22]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler

SC = StandardScaler()
SC.fit(matrix_X_train_mean_fv)
matrix_X_train_mean_fv_scaled = SC.transform(matrix_X_train_mean_fv)
matrix_X_test_mean_fv_scaled = SC.transform(matrix_X_test_mean_fv)

svc = SVC()
svc.fit(matrix_X_train_mean_fv_scaled, prova_train)

predictions = svc.predict(matrix_X_test_mean_fv_scaled)

print(accuracy_score(prova_test, predictions))

0.7866666666666666
